In [1]:
import cv2
import os

# Global variable to store the background
bg = None

# Function to compute the running average of the background
def run_avg(image, accumWeight):
    global bg
    # Initialize the background
    if bg is None:
        bg = image.copy().astype("float")
        return

    # Compute weighted average, accumulate it, and update the background
    cv2.accumulateWeighted(image, bg, accumWeight)

# Function to segment the hand region from the background
def segment(image, threshold=25):
    global bg
    # Find the absolute difference between background and current frame
    diff = cv2.absdiff(bg.astype("uint8"), image)

    # Threshold the difference image so that we get the foreground
    thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)[1]

    # Get the contours in the thresholded image
    (cnts, _) = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Return None if no contours detected
    if len(cnts) == 0:
        return
    else:
        # Based on contour area, get the maximum contour which is the hand
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)

# Set up webcam
cap = cv2.VideoCapture(0)

# Create directories for each gesture category
gesture_names = ['thumbs_up', 'thumbs_down', 'ok', 'fist', 'peace']

for gesture_name in gesture_names:
    os.makedirs(gesture_name, exist_ok=True)

# Parameters for background averaging
accumWeight = 0.5

# Initialize count variable
count = 0

# Capture and save hand gesture images
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply background averaging
    run_avg(gray, accumWeight)

    # Segment hand region
    hand_segment = segment(gray)

    # Check if hand segment is detected
    if hand_segment is not None:
        # Unpack thresholded image and segmented hand region
        (thresholded, segmented) = hand_segment

        # Draw contours on original frame
        cv2.drawContours(frame, [segmented], -1, (0, 255, 0), 2)

        # Display thresholded image
        cv2.imshow("Thresholded", thresholded)

        # Save captured frame and segmented hand region
        gesture_name = 'thumbs_up'  # Change gesture name as needed
        cv2.imwrite(os.path.join(gesture_name, f'frame_{count}.png'), frame)
        cv2.imwrite(os.path.join(gesture_name, f'segmented_{count}.png'), thresholded)

        # Increment count
        count += 1

    # Display captured frame
    cv2.imshow('Frame', frame)

    # Press 'q' to quit capturing
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release webcam and close windows
cap.release()
cv2.destroyAllWindows()

